In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import filters, transform, util
import os

In [ ]:
num_images = 100
c = np.random.choice(np.arange(592),100)

images = np.load('900C_driftcorrect.npy',mmap_mode='r+')[c]
print(images.shape)

In [ ]:
rotate = [90,180,270]
rotated_images = np.zeros((len(images)*len(rotate),images.shape[1],images.shape[2]))

n = 0
for i in range(len(images)):
    if i % 50 == 0:
        print('{} / {}'.format(i,len(images)))
    for j in rotate:
        rotated_images[n] = transform.rotate(images[i],j)
        n += 1

In [ ]:
all_images = np.concatenate((images,rotated_images),axis=0)
print(all_images.shape)

In [2]:
#np.save('900C_dataset_intermediate.npy',all_images)
all_images = np.load('900C_dataset_intermediate.npy',mmap_mode='r+')
print(all_images.shape)

(400, 3250, 3250)


In [3]:
#### 4096 **2 / 512 **2 = 64

all_image_sections = np.zeros((len(all_images) * 36,512 ,512))

n = 0
for i in range(len(all_images)):
    if i % 50 == 0:
        print(i)
    B = util.view_as_windows(all_images[i],window_shape=(512,512),step=512)
    B = B.reshape(-1,512,512)
    all_image_sections[n*36:n*36 + 36] = B
    n += 1

0
50
100
150
200
250
300
350


In [4]:
def scale(im):
    return (im - np.mean(im)) / np.std(im)

def norm(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

for i in range(len(all_image_sections)):
    all_image_sections[i] = norm(scale(all_image_sections[i]))
    if np.max(all_image_sections[i]) != 1 or np.min(all_image_sections[i]) != 0:
        print(i)

In [6]:
all_image_sections.shape
np.save('training_set.npy',all_image_sections)

In [5]:
for i in range(len(all_image_sections)):
    if np.max(all_image_sections[i]) != 1:
        print('i: ',i)
        print('max ',np.max(all_image_sections[i]))
    if np.min(all_image_sections[i]) != 0:
        print('min ',np.min(all_image_sections[i]))